### Settings

In [ ]:
import json
import pandas as pd

from visualize import *

my_dict = {'t1': 'T1','t2': 'T2','t3': 'T3','t4': 'T4','t5': 'T5',
           'v1': 'V1','v2': 'V2','v3': 'V3','v4': 'V4','v5': 'V5',
           'Zh':'Zh', 'Ko':'Ko', 'Es':'Es', 'Fr':'Fr', 'De':'De',
           'It':'It', 'Ar':'Ar', 'Ru':'Ru', 'Ja':'Ja', 'En':'En',
           'n':'Arabic Numeral', 'al':'Lowercase Latin', 'au':'Uppercase Latin',
           'rl':'Lowercase Roman', 'ru':'Uppercase Roman',
           'f':'Ascending', 'r':'Descending',
           'a person of routine and familiarity': 'Routine',
           'a more spontaneous and less reliable person': 'Spontaneous',
           'a person with reserved and lower energy levels': 'Reserved',
           "a competitive person, sometimes skeptical of others' intentions": 'Competitive',
           'a person with emotional stability and consistent moods': 'Stability',
           'an adventurous and creative person': 'Adventurous',
           'an organized person, mindful of details': 'Organized',
           'a person full of energy and positive emotions': 'Energy',
           'a compassionate and cooperative person': 'Compassionate',
           'a person with emotional instability and diverse negative feelings': 'Instability',
          }

my_colors = ['#e6194B', '#42d4f4', '#ffe119', '#3cb44b', '#f032e6', '#fabed4', '#469990', '#dcbeff',
             '#9A6324', '#fffac8', '#800000', '#aaffc3', '#000075', '#a9a9a9', '#000000']

markers = ["^", "o", "s", "p", "*", "P", "X", "h", "8", "d"]

default_color = '#D9DDDC'

def cstm_color(x, min_x, max_x):
    return plt.cm.plasma_r((np.clip(x,min_x,max_x)-min_x)/(max_x - min_x))

def get_colors(n):
    colors = ["white"] + [cstm_color(x, 1, n) for x in range(1,n+1)]
    return colors

### Load the saved documents

In [ ]:
# Section 3 - Default Experiments
chatgpt_data, info = extract_data('save/gpt-3.5-turbo-1106.json')
gpt4_data, _ = extract_data('save/gpt-4-1106.json')
gemini_data, _ = extract_data('save/gemini-1.0-pro.json')
llama_data, _ = extract_data('save/llama-3.1-8b.json')

# Section 4 - Representing Diverse Groups
## Create an Environment (low directive)
environment, _ = extract_data('save/environment.json')

## Embodying a Character (high directive)
character_cot, _ = extract_data('save/character_cot.json')
character_no, _ = extract_data('save/character.json')
character = pd.concat([character_cot, character_no], ignore_index=True)

## Assigning a Personality (moderate directive)
### Biography
biography_cot, _ = extract_data('save/personality_biography_cot.json')
biography_no, _ = extract_data('save/personality_biography.json')
biography = pd.concat([biography_cot, biography_no], ignore_index=True)
### Portray
portray_cot, _ = extract_data('save/personality_portray_cot.json')
portray_no, _ = extract_data('save/personality_portray.json')
portray = pd.concat([portray_cot, portray_no], ignore_index=True)
### Question Answering
qa_cot, _ = extract_data('save/personality_qa_cot.json')
qa_no, _ = extract_data('save/personality_qa.json')
qa = pd.concat([qa_cot, qa_no], ignore_index=True)
personality = pd.concat([biography, portray, qa], ignore_index=True)

# CoT
cot = pd.concat([character_cot, biography_cot, portray_cot, qa_cot])
no_cot = pd.concat([character_no, biography_no, portray_no, qa_no])

### Create PCA Reference

In [ ]:
conner = extract_reference('reference/conner.json')
reference = pd.concat([gemini_data, gpt4_data, llama_data, chatgpt_data], ignore_index=True)
reference = reference[["Openness","Conscientiousness","Extraversion","Agreeableness","Neuroticism"]]
reference = pd.concat([reference, conner], ignore_index=True)
vis = Visualize('BFI', reference)

### Plot Prompt Sensitivity Results

In [ ]:
models = [chatgpt_data, gpt4_data, gemini_data, llama_data]
model_names = ["gpt-3.5-turbo", "gpt-4-turbo", "gemini-1.0-pro", "llama-3.1-8b"]

for model_idx, model in enumerate(models):
    for aspect in info:
        vis.add(conner, color="white", marker=markers[4], alpha=0)
        for index, value in enumerate(model[aspect].unique()):
            vis.add(model[model[aspect] == value], my_colors[index], my_dict[value], marker=markers[index])
        vis.plot(f'{model_names[model_idx]}-{aspect}', format="svg", random_zorder=True, plot_reference=True, rotation_angle=52.5)
        vis.clean()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

models = [chatgpt_data, gpt4_data, gemini_data, llama_data]
model_names = ["gpt-3.5-turbo", "gpt-4-turbo", "gemini-1.0-pro", "llama-3.1-8b"]

def rotate(x, y, angle):
    angle = np.deg2rad(angle)
    x_rotated = x * np.cos(angle) - y * np.sin(angle)
    y_rotated = x * np.sin(angle) + y * np.cos(angle)
    return x_rotated, y_rotated

for index, model in enumerate(models):
    pca_data = vis.pca.transform(vis.extract(model))[:, :2]
    x, y = pca_data[:, 0], pca_data[:, 1]
    
    data_outliers = vis.detect_outlier(model, 0.30, 20)
    outliers_data = vis.pca.transform(vis.extract(data_outliers[model["Label"] == -1]))[:, :2]
    inliers_data = vis.pca.transform(vis.extract(data_outliers[model["Label"] != -1]))[:, :2]
    reference_data = vis.pca.transform(vis.extract(reference))[:, :2]

    pca_x, pca_y = rotate(pca_data[:, 0], pca_data[:, 1], 52.5)
    outlier_x, outlier_y = rotate(outliers_data[:, 0], outliers_data[:, 1], 52.5)
    inlier_x, inlier_y = rotate(inliers_data[:, 0], inliers_data[:, 1], 52.5)
    reference_x, reference_y = rotate(reference_data[:, 0], reference_data[:, 1], 52.5)

    smooth_gradient = LinearSegmentedColormap.from_list("smooth_gradient", ["white"] + get_colors(20))

    plt.style.use('default')
    plt.scatter(reference_x, reference_y, color="white", s=0, marker=markers[1])
    plt.scatter(outlier_x, outlier_y, color=my_colors[1], facecolors=(1, 1, 1, 0.1), marker=markers[0], label="Outliers")
    plt.scatter(inlier_x, inlier_y, color=my_colors[0], facecolors=(1, 1, 1, 0.1), marker=markers[1], label="Inliers")
    sns.kdeplot(x=pca_x, y=pca_y, cmap=smooth_gradient, fill=True, thresh=0, levels=100, alpha=0.5)    
    plt.xticks([])
    plt.yticks([])
    plt.xlim(-3, 5)
    plt.ylim(-2, 6)
    plt.legend(loc="upper left")
    plt.savefig(f'figures/{model_names[index]}-density.svg', format="svg", dpi=300)
    plt.clf()

In [ ]:
# Plot Environment Appending Tests
with open('dataset/environment.json') as f:
    env_conf = json.load(f)

for key in env_conf:
    vis.add(chatgpt_data, default_color, 'Default')
    for index, emotion in enumerate(env_conf[key].keys()):
        vis.add(environment[environment["append_label"] == emotion], my_colors[index+1], emotion.capitalize(), marker=markers[index+1])
    vis.plot(f"env-{key}", random_zorder=True, exclude=[0], format="svg", rotation_angle=52.5)
    vis.clean()

In [ ]:
# Plot Character Appending Tests
with open('dataset/character.json') as f:
    chara_conf = json.load(f)

for key in chara_conf:
    vis.add(chatgpt_data, default_color, 'Default')
    for index, chara in enumerate(chara_conf[key].keys()):
        vis.add(character[character["append_label"] == chara], my_colors[index], chara, marker=markers[index+1])
    vis.plot(f"chara-{key}", random_zorder=True, exclude=[0], format="svg", rotation_angle=52.5)
    vis.clean()
    

In [ ]:
# Plot Personality Tests
# Plot Personality Tests
personality_list = []
for file in ['personality_portray_cot', 'personality_portray', 'personality_biography_cot',
             'personality_biography', 'personality_qa_cot', 'personality_qa']:
    personality_list.append(extract_data(f'save/{file}.json')[0])
pers_data = pd.concat(personality_list)

with open('dataset/personality.json') as f:
    pers_conf = json.load(f)

for key in pers_conf:
    vis.add(chatgpt_data, default_color, 'Default')
    for index, pers in enumerate(pers_conf[key].keys()):
        vis.add(pers_data[pers_data["append_label"] == pers], my_colors[index], my_dict[pers].capitalize(), marker=markers[index+1])
    vis.plot(f"pers-{key}", random_zorder=True, exclude=[0], format="svg", rotation_angle=52.5)
    vis.clean()
   
 
# Plot Personalities Comparisons
for min_label, max_label in zip(list(pers_conf["minimum"].keys()), list(pers_conf["maximum"].keys())):
    vis.add(chatgpt_data, default_color, 'Default', marker=markers[0])
    vis.add(pers_data[pers_data["append_label"] == min_label], my_colors[0], my_dict[min_label].capitalize(), marker=markers[1])
    vis.add(pers_data[pers_data["append_label"] == max_label], my_colors[1], my_dict[max_label].capitalize(), marker=markers[2])
    vis.plot(f'{my_dict[min_label]}-{my_dict[max_label]}', exclude=[0], format="svg", random_zorder=True, rotation_angle=52.5)
    vis.clean()


In [ ]:
# Plot COT Results
for file in ['personality_portray', 'personality_biography', 'personality_qa', 'character']:
    cot, _ = extract_data(f'save/{file}_cot.json')
    no_cot, _ = extract_data(f'save/{file}.json')
    vis.add(chatgpt_data, default_color, 'Default', marker=markers[0])
    vis.add(cot, my_colors[0], 'With COT', marker=markers[1])
    vis.add(no_cot, my_colors[1], 'Without COT', marker=markers[2])
    vis.plot(f"cot-{file}", format="svg", random_zorder=True, exclude=[0], rotation_angle=52.5)
    vis.clean()
    